In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w9p1/train.csv
/kaggle/input/2024-ml-w9p1/test.csv
/kaggle/input/2024-ml-w9p1/sample_submit.csv


In [2]:
#데이터셋 로드
train = pd.read_csv("/kaggle/input/2024-ml-w9p1/train.csv")
test = pd.read_csv("/kaggle/input/2024-ml-w9p1/test.csv")
submit = pd.read_csv("/kaggle/input/2024-ml-w9p1/sample_submit.csv")

In [3]:
#target 정보 분리
y = train['target']
train = train.drop(['target'], axis=1)

In [4]:
#학습 데이터 확인
train.head()

,index,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36
1,1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47
2,3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52
3,4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8
4,5,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24


In [5]:
#데이터 삭제
train = train.drop(['index', 'enrollee_id', 'city'], axis=1)
test = test.drop(['index', 'enrollee_id', 'city'], axis=1)

In [6]:
#데이터타입 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  15326 non-null  float64
 1   gender                  11752 non-null  object 
 2   relevent_experience     15326 non-null  object 
 3   enrolled_university     15009 non-null  object 
 4   education_level         14964 non-null  object 
 5   major_discipline        13068 non-null  object 
 6   experience              15276 non-null  object 
 7   company_size            10612 non-null  object 
 8   company_type            10445 non-null  object 
 9   last_new_job            14983 non-null  object 
 10  training_hours          15326 non-null  int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3832 entries, 0 to 3831
Data columns (total 11 columns):
 #   Column                  Non-

In [7]:
#열 출력
train.columns

Index(['city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours'],
      dtype='object')

In [8]:
#데이터 실수화
from sklearn.preprocessing import LabelEncoder

x_data = pd.concat([train, test], axis=0)
col = ['gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job']

for i in col:
    le = LabelEncoder()
    le.fit(train[i])
    train[i] = le.transform(train[i])
    test[i] = le.transform(test[i])

In [9]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  15326 non-null  float64
 1   gender                  15326 non-null  int64  
 2   relevent_experience     15326 non-null  int64  
 3   enrolled_university     15326 non-null  int64  
 4   education_level         15326 non-null  int64  
 5   major_discipline        15326 non-null  int64  
 6   experience              15326 non-null  int64  
 7   company_size            15326 non-null  int64  
 8   company_type            15326 non-null  int64  
 9   last_new_job            15326 non-null  int64  
 10  training_hours          15326 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 1.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3832 entries, 0 to 3831
Data columns (total 11 columns):
 #   Column                  Non-Null Count 

In [10]:
#qda
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()

qda.fit(train, y)

QuadraticDiscriminantAnalysis()

In [11]:
#train 예측 및 정확도 출력
train_pred = qda.predict(train)

from sklearn.metrics import accuracy_score
print(accuracy_score(train_pred, y))

0.7696724520422811


In [12]:
#test 데이터 예측
test_pred = qda.predict(test)

In [13]:
#submit 저장
submit['target'] = test_pred.astype('int64')
submit.to_csv('submit.csv', index=False)

In [14]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3832 entries, 0 to 3831
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   index   3832 non-null   int64
 1   target  3832 non-null   int64
dtypes: int64(2)
memory usage: 60.0 KB
